In [1]:
import os

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import tensorflow as tf

%load_ext autoreload
%autoreload 2

In [3]:
from generate_input_planar_pose import GenerateInputPlanarPose

directory = os.path.expanduser('~/Documents/data/')

generator = GenerateInputPlanarPose([
    directory + 'cylinder-cube-1/cylinder-cube-1.db',
    directory + 'cylinder-1/cylinder-1b.db',
    directory + 'cube-1/cube-1.db',
    directory + 'cylinder-1/cylinder-1a.db',
], test_files=[
#    directory + 'all-1/all-1.db',
], output_folder='gen-inpaint/')

#generator.generateInput({
#    'did_grasp_weight': 0.13,
#    'force_rewrite': True,
#    'inpaint': True,
#    'size_cropped': (200, 200),
#    'size_input': (752, 480),
#    'size_output': (32, 32),
#})

In [10]:
from tensorflow_loader import DataLoader

test_loader = DataLoader(generator.test_output_filename)
train_loader = DataLoader(generator.train_output_filename, batch_size=128)

Length: 4870
Reward Mean: 0.20308008213552362
Length: 19233
Reward Mean: 0.20688400145583113


In [11]:
from tensorflow_utils import get_augmentation, single_class_split, tf_accuracy, tf_precision, tf_recall, tf_f1

tf.reset_default_graph()
# tf.set_random_seed(3)


train_dataset = tf.data.Dataset.from_generator(train_loader.nextBatch, (tf.float32, tf.float32), ((None, None, None, 1), (None, 3)))
train_dataset = train_dataset.map(get_augmentation(flip_up_down=False, flip_left_right=False, height=False, noise=False, defects=False, blur=False), num_parallel_calls=4)

test_dataset = tf.data.Dataset.from_generator(test_loader.entireBatch, (tf.float32, tf.float32))


handle = tf.placeholder(tf.string, shape=(), name='handle')
iterator = tf.data.Iterator.from_string_handle(handle, train_dataset.output_types, train_dataset.output_shapes)
train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()
    
image, label = iterator.get_next()


def act(x): return tf.nn.leaky_relu(x, 0.1)
def reg(l1=0.0, l2=0.3): return tf.contrib.layers.l1_l2_regularizer(l1, l2)
    
    
training = tf.placeholder_with_default(False, (), name='training')
apply_dropout = tf.placeholder_with_default(training, (), name='apply_dropout')
image = tf.identity(image, name='image')
label = tf.identity(label, name='label')


# Second layer with uncertainty about depth
#h = tf.to_float(tf.equal(image, 0.0))
#x = tf.concat([h, image], axis=3)
    
x = tf.layers.conv2d(image, 32, (5, 5), strides=(2, 2), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.batch_normalization(x, training=training)
#x = tf.layers.dropout(x, 0.4, training=apply_dropout)
    
x = tf.layers.conv2d(x, 48, (5, 5), strides=(1, 1), dilation_rate=(1, 1), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.batch_normalization(x, training=training)
#x = tf.layers.dropout(x, 0.4, training=apply_dropout)

x = tf.layers.conv2d(x, 64, (5, 5), dilation_rate=(1, 1), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.batch_normalization(x, training=training)
x = tf.layers.dropout(x, 0.4, training=apply_dropout)

x = tf.layers.conv2d(x, 142, (6, 6), dilation_rate=(1, 1), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
# x = tf.layers.batch_normalization(x, training=training)
x = tf.layers.dropout(x, 0.3, training=apply_dropout)
    
x = tf.layers.conv2d(x, 128, (1, 1), dilation_rate=(1, 1), activation=act, kernel_regularizer=reg(), bias_regularizer=reg())
x = tf.layers.dropout(x, 0.3, training=apply_dropout)
    
logits = tf.layers.conv2d(x, 3, (1, 1), dilation_rate=(1, 1), bias_regularizer=reg(l2=0.2))
prob = tf.nn.sigmoid(logits, name='prob')
# var_prob = tf.expand_dims(tf.abs(tf.reduce_mean(tf.reduce_mean(tf.gradients(prob, [x_depth])[0], axis=2, keepdims=True), axis=1, keepdims=True)), 3, name='var_prob')

    
reward, reward_pred = single_class_split(label, prob[:, 0, 0])
_, logits_pred = single_class_split(label, logits[:, 0, 0])

weights = tf.abs(0.6 - reward) # 1.0 or tf.abs(0.75 - reward) or label[:, 2]
loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=reward, logits=logits_pred, weights=weights)
loss_reg = loss + tf.add_n(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    
accuracy = tf_accuracy(reward, reward_pred)
precision = tf_precision(reward, reward_pred)
recall = tf_recall(reward, reward_pred)
f1 = tf_f1(precision, recall, beta=0.5)

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)  # For batch normalization layers
with tf.control_dependencies(extra_update_ops):
    train = tf.train.AdamOptimizer(learning_rate=1e-7).minimize(loss_reg, name='train')

In [13]:
from tensorflow_model import Model

model = Model(
    test_metrices=[loss, accuracy, precision, recall, f1],
    inputs={'image': image, 'apply_dropout': apply_dropout},
    outputs={'prob': prob},
    model_input_path=generator.model_directory + 'model-4-inpaint',
    model_output_path=generator.model_directory + 'model-4-inpaint'
)

model.fit(
    train_iterator, train_loader.batches_per_epoch, test_iterator,
    epochs=1500,
    early_stopping_patience=500,
    load=True,
    save=False,
    export=False,
)

INFO:tensorflow:Restoring parameters from /home/berscheid/Documents/data/cylinder-cube-1/models/model-4-inpaint
Epoch	Time[s]	Train Accuracy	Test Loss	Accuracy	Precision	Recall		F1
   -1	0.00	-1.0000		0.1265		0.8899		0.8168		0.5905		0.7586
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
    0	1.34	0.8984		0.1251		0.8918		0.8157		0.5976		0.7602
Best model yet!
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [ ]:
from tensorflow_model import Model

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

model = Model(
    test_metrices=[loss, accuracy, precision, recall, f1],
    model_input_path=generator.model_directory + 'model-4-2lyr'
)

train_handle = model.sess.run(train_iterator.string_handle())
model.sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
model.load()
model.sess.run(train_iterator.initializer)


batch_images = model.sess.run(image, feed_dict={handle: train_handle, training: True})
image_data = np.squeeze(batch_images[0], axis=2)

print(image_data.min(), image_data.max())
plt.imshow(image_data)

In [ ]:
# Analysis

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.rcParams['figure.figsize'] = (12, 9)

plt.grid(True)
plt.hist(prob_calc.flatten(), bins=np.arange(0, 1, 0.01))
plt.hist(test_labels[:, 0], bins=[0, 0.01, 0.99, 1], alpha=0.5)
plt.title('Histogram of the Grasp Reward Prediction of the Test Set')
plt.show()

In [ ]:
def largestErrorsIn(probs, ids, labels):
    # print(probs[np.arange(len(probs))
    diff = np.abs(probs - labels[:len(probs), 0])
    # diff = np.abs(probs[np.arange(len(probs)), labels[:, 1].astype(int)] - labels[:, 0])
    print('Mean diff: ', diff.mean())

    idxs = diff.argsort()[-200:][::-1]

    for idx in idxs:
        print()
        print('ID: ', ids[idx])
        print('Diff: ', diff[idx])
        print(probs[idx])
        print(labels[idx])
        
# largestErrorsIn(prob_calc, test_id, test_labels)
largestErrorsIn(np.array(train_predict), train_id, train_labels)

In [ ]:
for i in range(10, 20):
    print()
    print(test_id[-i])
    print(prob_calc[-i])
    print(test_labels[-i])
    img = mpimg.imread(test_image_names[-i])
    plt.figure()
    plt.title(test_id[-i])
    plt.imshow(img)